In [2]:
from collections import defaultdict
import pickle
from datetime import datetime, timedelta

import pandas as pd

### Chicago
Data source for Chicago:

https://www.chicago.gov/city/en/sites/covid-19/home/latest-data.html

Create a dict of the form {days from Feb 29: number of deaths}, and save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [52]:
chi_death_dict = defaultdict(int, {16: 1, 17: 1, 18: 1, 19: 2, 21: 1,
                                   22: 3, 23: 3, 24: 2, 25: 5, 26: 2,
                                   27: 7, 28: 4, 29: 21, 30: 11,
                                   31: 11, 32: 23, 33: 11, 34: 25,
                                   35: 19, 36: 28, 37: 15, 38: 22,
                                   39: 30, 40: 19, 41: 23, 42: 20,
                                   43: 24, 44: 12, 45: 1})

In [53]:
chi_last_day = max(chi_death_dict.keys())

In [54]:
chi_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + chi_last_day)]
chi_deaths =[chi_death_dict[day - 28] for day in range(29 + chi_last_day)]

In [55]:
chi_death_df = pd.DataFrame(chi_deaths, index=chi_index, columns=['deaths'])

In [56]:
# Inspect that last few days
chi_death_df.tail()

,deaths
2020-04-10,23
2020-04-11,20
2020-04-12,24
2020-04-13,12
2020-04-14,1


In [57]:
chi_death_df.to_pickle('../data/chi_deaths.pickle')

### Illinois
Data source for Illinois:

http://www.dph.illinois.gov/covid19/covid19-statistics

`il_cum_death_list` is the list of cumulative deaths on each day, starting on March 16. Again, save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [43]:
il_cum_death_list = [0, 1, 3, 4, 5, 6, 9, 12, 16, 19, 26, 34, 47,
                     65, 72, 99, 141, 157, 210, 243, 274, 307, 380,
                     462, 528, 596, 677, 720, 794, 868]

In [44]:
# New deaths from March 17
il_death_list = [now - prev 
                 for now, prev in zip(il_cum_death_list[1:],
                                      il_cum_death_list[:-1])]

In [45]:
il_death_dict = defaultdict(int, {index + 17: diff
                                  for index, diff in enumerate(il_death_list)})

In [46]:
il_num_days = len(il_death_list)

In [47]:
num_days = len(il_death_list)

In [48]:
il_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + 16 + num_days)]
il_deaths =[il_death_dict[day - 28] for day in range(29 + 16 + num_days)]

In [49]:
il_death_df = pd.DataFrame(il_deaths, index=il_index, columns=['deaths'])

In [50]:
# Inspect that last few days
il_death_df.tail()

,deaths
2020-04-10,68
2020-04-11,81
2020-04-12,43
2020-04-13,74
2020-04-14,74


In [51]:
il_death_df.to_pickle('../data/il_deaths.pickle')